In [1]:
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Cleaning of master 2021-2023

In [18]:
import pandas as pd
import re

#read the csv into colab
with open('sample_data/Master_2021-2023.csv', encoding='utf-8', errors='ignore') as f:
  raw_df = pd.read_csv(f).drop_duplicates()
  print(len(raw_df))

# List of dataframes to be added together later
dfs = []

# Extract categories from column titles
categories = [category for category in raw_df.columns if "Posts in" in category]

# Iterate through categories, creating seperate dataframes for each, adding to dfs then concatinate
for category in categories:

  # Select the rows with responses in the given category
  category_df = raw_df[category][raw_df[category].notnull()]

  # If the category has no responses skip that category
  if len(category_df) == 0:
    continue

  # Select the columns that won't be transformed
  first_4_columns = raw_df[['Mentee ID','Mentor ID','Mentor Created at', 'Relationship ID']]

  # Split in the format Mentor/Mentee, date and time, response, seperating into different cells.
  split_df = category_df.str.split(pat= r'(Mentor|Mentee)(\s+\d{4}\-\d{2}\-\d{2},\s\d{2}:\d{2}):\s', n=None, regex=True, expand=True)

  # If number of columns is 1, then the responses are likely in the alternative format
  if len(split_df.columns) == 1:
    split_df = category_df.str.split(pat= r'([Mm]entor|[Mm]entee)\scommented\sat\s(\d?\d:\d{2}[AP]M\s.*\s\d?\d)', n=None, regex=True, expand=True)

  # Recombine the transoformed and non-transformed columns, droping na values
  concat_df = pd.concat([first_4_columns, split_df], axis=1).dropna(subset=[0])

  # Select the columns that will be put into "Mentor", "Reponse Datetime", and "Response" columns.
  mentor_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][::3]
  date_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][1::3]
  response_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][2::3]

  # Separate into dataframes each containing one of the new columns and the non-transformed columns
  response_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=response_cols, var_name='Response_Col', value_name='Response')
  date_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=date_cols, var_name='date_Col', value_name='Response Datetime')
  mentor_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=mentor_cols, var_name='mentor_Col', value_name='Mentor')

  # Recombine all of the dataframes, drop 'date_Col', na values, and duplicates
  joined_df = pd.concat([date_df, response_df['Response'], mentor_df['Mentor']], axis=1)
  joined_df = joined_df.drop(columns=['date_Col']).dropna().drop_duplicates()

  # Add category column and set to current category
  joined_df['Category'] = category

  # Add dataframe to dfs
  dfs.append(joined_df)

# Combine all category dataframes
result_df_2021_2023 = pd.concat(dfs)

# Transform columns to appropriate types, striping white space, and capitalizing Mentor column
result_df_2021_2023['Mentor Created at'] = pd.to_datetime(result_df_2021_2023['Mentor Created at'])
result_df_2021_2023['Response Datetime'] = pd.to_datetime(result_df_2021_2023['Response Datetime'], format='mixed')
result_df_2021_2023['Response'] = result_df_2021_2023['Response'].str.strip().dropna()
result_df_2021_2023['Mentor'] = result_df_2021_2023['Mentor'].str.capitalize()

# Save as csv file
file_path = 'clean_data/Master_2021-2023_Cleaned.csv'
result_df_2021_2023.to_csv(file_path, index=False)
result_df_2021_2023


76


,Mentor ID,Mentee ID,Mentor Created at,Relationship ID,Response Datetime,Response,Mentor,Category
0,1047627114,1047627192,2023-09-18 09:42:00,67068,2023-11-06 09:28:00,Do you think there is a difference between lea...,Mentee,Posts in Ways of Knowing
1,1047587166,1047627570,2022-11-10 13:57:00,67331,2023-10-11 16:06:00,"hey sorry, i just got broken up with and shits...",Mentee,Posts in Ways of Knowing
2,1047627114,1047627192,2023-09-18 09:42:00,67068,2023-11-09 20:48:00,"Sarah, I 100% agree on your take about the dif...",Mentor,Posts in Ways of Knowing
3,1047587166,1047627570,2022-11-10 13:57:00,67331,2023-10-22 18:37:00,"Hi Aspen, I'm so sorry to hear about the break...",Mentor,Posts in Ways of Knowing
4,1047627114,1047627192,2023-09-18 09:42:00,67068,2023-11-14 10:22:00,"Well, right now, I'm in B.C First Peoples, and...",Mentee,Posts in Ways of Knowing
...,...,...,...,...,...,...,...,...
6,1047555384,1047628803,2022-03-07 13:35:00,68123,2024-04-08 09:21:00,,Mentee,Posts in Getting Hired
7,1047582324,1047627553,2022-09-15 15:42:00,67334,2024-01-16 15:41:00,"Hi Ellie, thank you for answering the question...",Mentor,Posts in Getting Hired
8,1047555384,1047628803,2022-03-07 13:35:00,68123,2024-04-10 16:22:00,"Hey Jade, I'm sorry I'm not sure if there's a ...",Mentor,Posts in Getting Hired
9,1047582324,1047627553,2022-09-15 15:42:00,67334,2024-01-18 23:01:00,"Hi Genevieve, thank you for being my mentor! I...",Mentee,Posts in Getting Hired


### Cleaning of master 2021_2022 through April

In [19]:
## Adapted Code from Jonah's

import pandas as pd
import re

#read the csv into colab
with open('sample_data/Master2021_2022.csv', encoding='utf-8', errors='ignore') as f:
  raw_df_2021_2022 = pd.read_csv(f).drop_duplicates()
  ## Rename the column  'Relation[Created At]' to 'Mentor Created at' to match the other datasets.
  raw_df_2021_2022.rename(columns={'Relation[Created At]': 'Relation Created at'}, inplace=True) 
  print(len(raw_df_2021_2022))

# List of dataframes to be added together later
dfs_2021_2022 = []

# Extract categories from column titles
categories_2021_2022 = [category_ for category_ in raw_df_2021_2022.columns if "Posts in" in category_]

# Iterate through categories, creating seperate dataframes for each, adding to dfs then concatinate
for category_ in categories_2021_2022 :

  # Select the rows with responses in the given category
  category_df = raw_df_2021_2022[category_][raw_df_2021_2022[category_].notnull()]

  # If the category has no responses skip that category
  if len(category_df) == 0:
    continue

  # Select the columns that won't be transformed
  first_4_columns = raw_df_2021_2022[['Mentor ID', 'Mentee ID', 'Relation Created at']]

  # Split in the format Mentor/Mentee, date and time, response, seperating into different cells.
  split_df = category_df.str.split(pat= r'(Mentor|Mentee)(\s+\d{4}\-\d{2}\-\d{2},\s\d{2}:\d{2}):\s', n=None, regex=True, expand=True)

  # If number of columns is 1, then the responses are likely in the alternative format
  if len(split_df.columns) == 1:
    split_df = category_df.str.split(pat= r'([Mm]entor|[Mm]entee)\scommented\sat\s(\d?\d:\d{2}[AP]M\s.*\s\d?\d)', n=None, regex=True, expand=True)

  # Recombine the transoformed and non-transformed columns, droping na values
  concat_df = pd.concat([first_4_columns, split_df], axis=1).dropna(subset=[0])

  # Select the columns that will be put into "Mentor", "Reponse Datetime", and "Response" columns.
  mentor_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Relation Created at',0])][::3]
  date_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Relation Created at',0])][1::3]
  response_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Relation Created at',0])][2::3]

  # Separate into dataframes each containing one of the new columns and the non-transformed columns
  response_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Relation Created at'], value_vars=response_cols, var_name='Response_Col', value_name='Response')
  date_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Relation Created at'], value_vars=date_cols, var_name='date_Col', value_name='Response Datetime')
  mentor_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Relation Created at'], value_vars=mentor_cols, var_name='mentor_Col', value_name='Mentor')

  # Recombine all of the dataframes, drop 'date_Col', na values, and duplicates
  joined_df = pd.concat([date_df, response_df['Response'], mentor_df['Mentor']], axis=1)
  joined_df = joined_df.drop(columns=['date_Col']).dropna().drop_duplicates()

  # Add category column and set to current category
  joined_df['Category'] = category_

  # Add dataframe to dfs
  dfs_2021_2022.append(joined_df)

# Combine all category dataframes
result_df_2021_2022 = pd.concat(dfs_2021_2022)

# Transform columns to appropriate types, striping white space, and capitalizing Mentor column
result_df_2021_2022['Relation Created at'] = pd.to_datetime(result_df_2021_2022['Relation Created at'])
result_df_2021_2022['Response Datetime'] = pd.to_datetime(result_df_2021_2022['Response Datetime'], format='mixed')
result_df_2021_2022['Response'] = result_df_2021_2022['Response'].str.strip().dropna()
result_df_2021_2022['Mentor'] = result_df_2021_2022['Mentor'].str.capitalize()

## Adding the relationship ID column 

#read the IDs csv file
with open('sample_data/ids_2021_2022.csv', encoding='utf-8', errors='ignore') as f:
  ids_df_2021_2022 = pd.read_csv(f).drop_duplicates()

merged_df = pd.merge(result_df_2021_2022, ids_df_2021_2022, on=['Mentor ID', 'Mentee ID'], how='left')
# Reorder columns to have 'Relationship ID' in the fourth position
cols = list(merged_df.columns)
cols.insert(3, cols.pop(cols.index('Relationship ID')))
result_df_2021_2022 = merged_df[cols]

result_df_2021_2022.sort_values(by='Response Datetime')

# Save as csv file
file_path = 'clean_data/Master2021_2022_cleaned.csv'
result_df_2021_2022.to_csv(file_path, index=False)
result_df_2021_2022


325


,Mentor ID,Mentee ID,Relation Created at,Relationship ID,Response Datetime,Response,Mentor,Category
0,1047550236,1047554192,2022-02-18 19:15:00,48120,2022-03-12 14:23:00,"Hi Lukas, I found this documentary intriguing,...",Mentor,"Posts in {""en""=>""Finding\nInspiration""}"
1,1047547565,1047554190,2022-02-10 18:48:00,47954,2022-03-08 19:23:00,"Hi Chantel, How are you doing? I hope you alre...",Mentor,"Posts in {""en""=>""Finding\nInspiration""}"
2,1047550342,1047554102,2022-02-09 20:34:00,47928,2022-02-20 07:40:00,"Hi Trisha, Finding Inspiration is unlocked now...",Mentor,"Posts in {""en""=>""Finding\nInspiration""}"
3,1047514152,1047554099,2022-02-09 20:33:00,47927,2022-02-22 21:35:00,The quote that stood out to me was “Not knowin...,Mentee,"Posts in {""en""=>""Finding\nInspiration""}"
4,1047546936,1047554098,2022-02-09 20:32:00,47926,2022-02-18 15:51:00,Hi Piper! I'm excited to hear about who or wha...,Mentor,"Posts in {""en""=>""Finding\nInspiration""}"
...,...,...,...,...,...,...,...,...
5352,1047548830,1047554042,2022-02-09 18:21:00,47895,2022-04-12 14:05:00,Hi\nErica! I feel that self-care is very impor...,Mentee,"Posts in {:en=>""Complete\nConversation""} Rural..."
5353,1047514148,1047541039,2021-09-22 21:12:00,40652,2022-01-27 02:24:00,I feel like it is time to say goodbye lol I am...,Mentee,"Posts in {:en=>""Complete\nConversation""} Payin..."
5354,1047516507,1047538825,2021-09-14 12:55:00,40141,2021-10-12 22:06:00,"Once you're ready to move on, let me know whic...",Mentor,"Posts in {:en=>""Complete\nConversation""} Payin..."
5355,1047514148,1047541039,2021-09-22 21:12:00,40652,2022-01-31 00:02:00,I am happy to hear that our conversations were...,Mentor,"Posts in {:en=>""Complete\nConversation""} Payin..."


### Cleaning of 2022-2023SY

In [20]:
###sample_data/Master_2022-2023SY.csv
import pandas as pd
import re

#read the csv into colab
with open('sample_data/Master_2022-2023SY.csv', encoding='utf-8', errors='ignore') as f:
  raw_df_2022_2023SY = pd.read_csv(f).drop_duplicates()
  print(len(raw_df_2022_2023SY))

# List of dataframes to be added together later
dfs_2022_2023 = []

# Extract categories from column titles
categories_2022_2023 = [category for category in raw_df_2022_2023SY.columns if "Posts in" in category]

# Iterate through categories, creating seperate dataframes for each, adding to dfs then concatinate
for category in categories_2022_2023:

  # Select the rows with responses in the given category
  category_df = raw_df_2022_2023SY[category][raw_df_2022_2023SY[category].notnull()]

  # If the category has no responses skip that category
  if len(category_df) == 0:
    continue

  # Select the columns that won't be transformed
  first_4_columns = raw_df_2022_2023SY[['Mentee ID','Mentor ID','Mentor Created at', 'Relationship ID']]

  # Split in the format Mentor/Mentee, date and time, response, seperating into different cells.
  split_df = category_df.str.split(pat= r'(Mentor|Mentee)(\s+\d{4}\-\d{2}\-\d{2},\s\d{2}:\d{2}):\s', n=None, regex=True, expand=True)

  # If number of columns is 1, then the responses are likely in the alternative format
  if len(split_df.columns) == 1:
    split_df = category_df.str.split(pat= r'([Mm]entor|[Mm]entee)\scommented\sat\s(\d?\d:\d{2}[AP]M\s.*\s\d?\d)', n=None, regex=True, expand=True)

  # Recombine the transoformed and non-transformed columns, droping na values
  concat_df = pd.concat([first_4_columns, split_df], axis=1).dropna(subset=[0])

  # Select the columns that will be put into "Mentor", "Reponse Datetime", and "Response" columns.
  mentor_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][::3]
  date_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][1::3]
  response_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][2::3]

  # Separate into dataframes each containing one of the new columns and the non-transformed columns
  response_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=response_cols, var_name='Response_Col', value_name='Response')
  date_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=date_cols, var_name='date_Col', value_name='Response Datetime')
  mentor_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=mentor_cols, var_name='mentor_Col', value_name='Mentor')

  # Recombine all of the dataframes, drop 'date_Col', na values, and duplicates
  joined_df = pd.concat([date_df, response_df['Response'], mentor_df['Mentor']], axis=1)
  joined_df = joined_df.drop(columns=['date_Col']).dropna().drop_duplicates()

  # Add category column and set to current category
  joined_df['Category'] = category

  # Add dataframe to dfs
  dfs_2022_2023.append(joined_df)

# Combine all category dataframes
result_df_2022_2023SY = pd.concat(dfs_2022_2023)

# Transform columns to appropriate types, striping white space, and capitalizing Mentor column
result_df_2022_2023SY['Mentor Created at'] = pd.to_datetime(result_df_2022_2023SY['Mentor Created at'])
result_df_2022_2023SY['Response Datetime'] = pd.to_datetime(result_df_2022_2023SY['Response Datetime'], format='mixed')
result_df_2022_2023SY['Response'] = result_df_2022_2023SY['Response'].str.strip().dropna()
result_df_2022_2023SY['Mentor'] = result_df_2022_2023SY['Mentor'].str.capitalize()

# Save as csv file
file_path = 'clean_data/Master_2022_2023SY_Cleaned.csv'
result_df_2022_2023SY.to_csv(file_path, index=False)
result_df_2022_2023SY


297


,Mentor ID,Mentee ID,Mentor Created at,Relationship ID,Response Datetime,Response,Mentor,Category
0,1047567826,1047594893,2022-08-25 15:21:00,59937,2023-03-16 16:36:00,"Hello Michael, My name is Syed and I will be y...",Mentor,Posts in Ways of Knowing
1,1047582928,1047594513,2022-09-21 14:32:00,59815,2023-05-16 11:15:00,"Hi Zee, This video about Hauyat was really nea...",Mentee,Posts in Ways of Knowing
2,1047549910,1047594022,2021-10-30 13:51:00,59577,2023-04-17 09:11:00,when studying a land you don't have the same c...,Mentee,Posts in Ways of Knowing
3,1047584290,1047593971,2022-10-11 13:49:00,59523,2023-04-26 13:41:00,"Hey Kaiden, no worries about the delayed respo...",Mentor,Posts in Ways of Knowing
4,1047582928,1047592065,2022-09-21 14:32:00,58840,2023-05-23 11:39:00,"Hi Jayden, It's opened up! I'm excited to hear...",Mentor,Posts in Ways of Knowing
...,...,...,...,...,...,...,...,...
159,1047582235,1047582372,2022-09-14 22:19:00,54546,2022-11-15 13:56:00,As someone with indigenous with heritage it is...,Mentee,Posts in Confronting Discrimination
172,1047549501,1047592059,2021-10-21 20:08:00,58834,2023-05-30 08:47:00,yeah he acknowledged that what he did was wron...,Mentee,Posts in Confronting Discrimination
177,1047582235,1047582372,2022-09-14 22:19:00,54546,2022-11-16 08:57:00,Yeah. How do you think her experiences have sh...,Mentor,Posts in Confronting Discrimination
195,1047582235,1047582372,2022-09-14 22:19:00,54546,2022-11-17 15:55:00,Fair assessment. Do you have anything else you...,Mentor,Posts in Confronting Discrimination


### Cleaning of the file 2022 spring

In [21]:
##Master_2022_spring.csv

import pandas as pd
import re

#read the csv into colab
with open('sample_data/Master_2022_spring.csv', encoding='utf-8', errors='ignore') as f:
  raw_df_2022_spring = pd.read_csv(f).drop_duplicates()
  print(len(raw_df_2022_spring))

# List of dataframes to be added together later
dfs_2022_spring = []

# Extract categories from column titles
categories_2022_spring = [category for category in raw_df_2022_spring.columns if "Posts in" in category]

# Iterate through categories, creating seperate dataframes for each, adding to dfs then concatinate
for category in categories_2022_spring:

  # Select the rows with responses in the given category
  category_df = raw_df_2022_spring[category][raw_df_2022_spring[category].notnull()]

  # If the category has no responses skip that category
  if len(category_df) == 0:
    continue

  # Select the columns that won't be transformed
  first_4_columns = raw_df_2022_spring[['Mentee ID','Mentor ID','Mentor Created at', 'Relationship ID']]

  # Split in the format Mentor/Mentee, date and time, response, seperating into different cells.
  split_df = category_df.str.split(pat= r'(Mentor|Mentee)(\s+\d{4}\-\d{2}\-\d{2},\s\d{2}:\d{2}):\s', n=None, regex=True, expand=True)

  # If number of columns is 1, then the responses are likely in the alternative format
  if len(split_df.columns) == 1:
    split_df = category_df.str.split(pat= r'([Mm]entor|[Mm]entee)\scommented\sat\s(\d?\d:\d{2}[AP]M\s.*\s\d?\d)', n=None, regex=True, expand=True)

  # Recombine the transoformed and non-transformed columns, droping na values
  concat_df = pd.concat([first_4_columns, split_df], axis=1).dropna(subset=[0])

  # Select the columns that will be put into "Mentor", "Reponse Datetime", and "Response" columns.
  mentor_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][::3]
  date_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][1::3]
  response_cols = concat_df.columns[~concat_df.columns.isin(['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID',0])][2::3]

  # Separate into dataframes each containing one of the new columns and the non-transformed columns
  response_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=response_cols, var_name='Response_Col', value_name='Response')
  date_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=date_cols, var_name='date_Col', value_name='Response Datetime')
  mentor_df = pd.melt(concat_df, id_vars=['Mentor ID', 'Mentee ID', 'Mentor Created at', 'Relationship ID'], value_vars=mentor_cols, var_name='mentor_Col', value_name='Mentor')

  # Recombine all of the dataframes, drop 'date_Col', na values, and duplicates
  joined_df = pd.concat([date_df, response_df['Response'], mentor_df['Mentor']], axis=1)
  joined_df = joined_df.drop(columns=['date_Col']).dropna().drop_duplicates()

  # Add category column and set to current category
  joined_df['Category'] = category

  # Add dataframe to dfs
  dfs_2022_spring.append(joined_df)

# Combine all category dataframes
result_df_2022_spring = pd.concat(dfs_2022_spring)

# Transform columns to appropriate types, striping white space, and capitalizing Mentor column
result_df_2022_spring['Mentor Created at'] = pd.to_datetime(result_df_2022_spring['Mentor Created at'])
result_df_2022_spring['Response Datetime'] = pd.to_datetime(result_df_2022_spring['Response Datetime'], format='mixed')
result_df_2022_spring['Response'] = result_df_2022_spring['Response'].str.strip().dropna()
result_df_2022_spring['Mentor'] = result_df_2022_spring['Mentor'].str.capitalize()

# Save as csv file
file_path = 'clean_data/Master_2022_spring_Cleaned.csv'
result_df_2022_spring.to_csv(file_path, index=False)
result_df_2022_spring


36


,Mentor ID,Mentee ID,Mentor Created at,Relationship ID,Response Datetime,Response,Mentor,Category
0,1047516499,1047540775,2020-11-27 16:54:00,40437,2022-03-09 13:49:00,"Hi Rachel, Thanks for your understanding and p...",Mentor,Posts in Ways of Knowing
1,1047516499,1047540775,2020-11-27 16:54:00,40437,2022-03-30 21:41:00,"Hello, I really hope you get lots of job offer...",Mentee,Posts in Ways of Knowing
2,1047516499,1047540775,2020-11-27 16:54:00,40437,2022-04-10 18:55:00,"Hi Rachel, OMG. First and foremost, I am soooo...",Mentor,Posts in Ways of Knowing
3,1047516499,1047540775,2020-11-27 16:54:00,40437,2022-04-18 20:10:00,"Hi, I'm happy your back. Sorry about your old ...",Mentee,Posts in Ways of Knowing
4,1047516499,1047540775,2020-11-27 16:54:00,40437,2022-04-21 18:35:00,"Hi Rachel, That's amazing! That's my mentee, e...",Mentor,Posts in Ways of Knowing
...,...,...,...,...,...,...,...,...
1,10161,1047561227,2017-01-10 17:16:00,51050,2024-10-07 20:40:00,No worries! Hope you had a good long weekend!!...,Mentor,Posts in Getting Hired
2,10161,1047561227,2017-01-10 17:16:00,51050,2024-10-05 18:42:00,"Hello! Sorry for the late reply, my family and...",Mentee,Posts in Getting Hired
3,10161,1047561227,2017-01-10 17:16:00,51050,2024-10-02 12:28:00,Okay that sounds great! Looking forward to hea...,Mentor,Posts in Getting Hired
4,10161,1047561227,2017-01-10 17:16:00,51050,2024-09-29 18:58:00,"Hello and thanks! And yes, I agree! I think it...",Mentee,Posts in Getting Hired


### MERGING THE DATA (ATTEMPT)

In [22]:
## Combine the 3 datasets that have the same columns
combined_df = pd.concat([result_df_2021_2023, result_df_2022_2023SY, result_df_2022_spring])

## Drop "Mentor Created at" from Combined and drop "Relation Created at" from result_df_2021_2022.
## We will use their first response/text as relation created at!!!
combined_df = combined_df.drop(columns=['Mentor Created at'])
result_df_2021_2022 = result_df_2021_2022.drop(columns=['Relation Created at'])

## Combine all 4 dataframes together
final_combined_df = pd.concat([combined_df, result_df_2021_2022])
final_combined_df.drop_duplicates()

,Mentor ID,Mentee ID,Relationship ID,Response Datetime,Response,Mentor,Category
0,1047627114,1047627192,67068,2023-11-06 09:28:00,Do you think there is a difference between lea...,Mentee,Posts in Ways of Knowing
1,1047587166,1047627570,67331,2023-10-11 16:06:00,"hey sorry, i just got broken up with and shits...",Mentee,Posts in Ways of Knowing
2,1047627114,1047627192,67068,2023-11-09 20:48:00,"Sarah, I 100% agree on your take about the dif...",Mentor,Posts in Ways of Knowing
3,1047587166,1047627570,67331,2023-10-22 18:37:00,"Hi Aspen, I'm so sorry to hear about the break...",Mentor,Posts in Ways of Knowing
4,1047627114,1047627192,67068,2023-11-14 10:22:00,"Well, right now, I'm in B.C First Peoples, and...",Mentee,Posts in Ways of Knowing
...,...,...,...,...,...,...,...
5352,1047548830,1047554042,47895,2022-04-12 14:05:00,Hi\nErica! I feel that self-care is very impor...,Mentee,"Posts in {:en=>""Complete\nConversation""} Rural..."
5353,1047514148,1047541039,40652,2022-01-27 02:24:00,I feel like it is time to say goodbye lol I am...,Mentee,"Posts in {:en=>""Complete\nConversation""} Payin..."
5354,1047516507,1047538825,40141,2021-10-12 22:06:00,"Once you're ready to move on, let me know whic...",Mentor,"Posts in {:en=>""Complete\nConversation""} Payin..."
5355,1047514148,1047541039,40652,2022-01-31 00:02:00,I am happy to hear that our conversations were...,Mentor,"Posts in {:en=>""Complete\nConversation""} Payin..."


#### Final clean up

In [23]:
# Sort the combined_df by "Response Datetime" to ensure the data is in the correct order
final_combined_df_sorted = final_combined_df.sort_values('Response Datetime')

# Find the first occurrence of each pair "Mentor ID", "Mentee ID" based on "Response Datetime"
first_response = final_combined_df_sorted.groupby(['Mentor ID', 'Mentee ID'])['Response Datetime'].min().reset_index()

# Merge the first_response DataFrame with combined_df based on "Mentor ID" and "Mentee ID"
final_combined_df_sorted = final_combined_df_sorted.merge(first_response, on=['Mentor ID', 'Mentee ID'], suffixes=('', '_first'), how='left')

# Rename the "Response Datetime_first" column to "Relation Created at" and reorder the columns
final_combined_df_sorted.rename(columns={'Response Datetime_first': 'Relation Created at'}, inplace=True)
columns = final_combined_df_sorted.columns.tolist()
columns.insert(2, columns.pop(columns.index('Relation Created at')))
final_combined_df_sorted = final_combined_df_sorted[columns]

final_combined_df_sorted = final_combined_df_sorted.drop_duplicates()
final_combined_df_sorted['Response'] = final_combined_df_sorted['Response'].str.replace('\n', ' ')
final_combined_df_sorted['Category'] = final_combined_df_sorted['Category'].str.replace('\n', ' ')


# Save as csv file
file_path = 'clean_data/Master_all_Cleaned.csv'
final_combined_df_sorted = final_combined_df.sort_values('Response Datetime')
final_combined_df_sorted.to_csv(file_path, index=False)
final_combined_df_sorted.drop_duplicates()

,Mentor ID,Mentee ID,Relationship ID,Response Datetime,Response,Mentor,Category
2845,1047517758,1047538818,40134,2021-09-14 14:46:00,Hi Allan! My name is Lillie and I am a fourth ...,Mentor,"Posts in {""en""=>""Program\nInstructions""}"
2842,1047512489,1047538891,40137,2021-09-14 16:05:00,"Hello, I'm Eden, I use he/they pronouns and I ...",Mentee,"Posts in {""en""=>""Program\nInstructions""}"
2846,1047516311,1047538822,40133,2021-09-14 16:07:00,Hi,Mentee,"Posts in {""en""=>""Program\nInstructions""}"
2843,1047516309,1047540777,40136,2021-09-14 16:07:00,Hi Colleen! My name is Khalifa and I'm going t...,Mentor,"Posts in {""en""=>""Program\nInstructions""}"
2840,1047499271,1047538812,40139,2021-09-14 16:16:00,Hi,Mentee,"Posts in {""en""=>""Program\nInstructions""}"
...,...,...,...,...,...,...,...
4,10161,1047561227,51050,2024-09-29 18:58:00,"Hello and thanks! And yes, I agree! I think it...",Mentee,Posts in Getting Hired
3,10161,1047561227,51050,2024-10-02 12:28:00,Okay that sounds great! Looking forward to hea...,Mentor,Posts in Getting Hired
2,10161,1047561227,51050,2024-10-05 18:42:00,"Hello! Sorry for the late reply, my family and...",Mentee,Posts in Getting Hired
1,10161,1047561227,51050,2024-10-07 20:40:00,No worries! Hope you had a good long weekend!!...,Mentor,Posts in Getting Hired


### Load the docx files into the dataframes

In [24]:
import pandas as pd

# List of relationship IDs
relation_IDs = [40437, 51050, 55268, 59057, 59809, 64642, 67329]

# Initialize an empty DataFrame to store all data
docx_data = pd.DataFrame()

for rel_id in relation_IDs:
    # Read the CSV file
    file_path = f'clean_data/{rel_id}.csv'
    df = pd.read_csv(file_path)
    
    # Add Relationship ID
    df['Relationship ID'] = rel_id
    
    # Convert Date to datetime format
    if 'Date' in df.columns:
        df['Response Datetime'] = pd.to_datetime(df['Date'] + "-2022", format='%I:%M%p %B %d-%Y', errors='coerce')
        df.drop(columns=['Date'], inplace=True)
    
    # Rename columns
    df.rename(columns={'Person': 'Mentor', 'Text': 'Response'}, inplace=True)
    
    # Reorder columns if needed
    df = df[['Relationship ID', 'Response Datetime', 'Response', 'Mentor']]
    
    # Remove " commented at" from Mentor column if needed
    df['Mentor'] = df['Mentor'].str.replace(r' commented at', '', regex=True)
    
    # Remove timestamps from Response
    df['Response'] = df['Response'].str.replace(r'^\d+:\d+\w{2}\s\w+\s\d+', '', regex=True)
    
    # Append the processed data to the all_data DataFrame
    docx_data = pd.concat([docx_data, df], ignore_index=True)


docx_data

,Relationship ID,Response Datetime,Response,Mentor
0,40437,2022-01-08 20:53:00,"\nHi Mentee,\nIs that... ✨ growth ✨ I hear? I...",Mentor
1,40437,2022-01-06 10:25:00,\nHello! Happy New Year!\nI think the first f...,Mentee
2,40437,2022-11-29 21:03:00,\nHi Mentee!\nWELCOME BACK TO MENTORING TO TH...,Mentor
3,40437,2022-11-24 09:21:00,"\nHello!\nOkay, so it's the end of November.....",Mentee
4,40437,2022-09-17 22:59:00,"\nHi Mentee,\n$15 for a software you can appl...",Mentor
...,...,...,...,...
681,67329,2022-09-23 12:27:00,"\nHi Hi Hi,\n \nAs I am writing this it's 1:4...",Mentee
682,67329,2022-09-22 17:32:00,"\nHi Mentee!\nThis is actually so cool, we sh...",Mentor
683,67329,2022-09-21 14:54:00,"\nFor this first image, I chose a picture fro...",Mentee
684,67329,2022-09-21 14:31:00,\nHi Mentor! Nice to meet you! My name is Men...,Mentee


#### ASSIGN MENTOR ID AND MENTEE ID TO docx data 

In [25]:
### Assign the Mentor ID and mentee ID
docx_data_with_ids = pd.merge(docx_data, final_combined_df_sorted[['Mentor ID', 'Mentee ID', 'Relationship ID']], on='Relationship ID', how='left')
docx_data_with_ids = docx_data_with_ids.reindex(columns=['Mentor ID', 'Mentee ID', 'Relationship ID', 'Response Datetime', 'Response', 'Mentor'])
docx_data_with_ids = docx_data_with_ids.drop_duplicates()

# Save the docx data DataFrame as a CSV file
docx_file_path = 'clean_data/docx_data.csv'
docx_data_with_ids.to_csv(docx_file_path, index=False)

In [ ]:
# # Save the final combined DataFrame as a CSV file
# final_combined_df_sorted  = pd.concat([final_combined_df_sorted, docx_data], ignore_index=True)
# final_combined_df_sorted = final_combined_df.sort_values('Response Datetime')
# final_combined_df_sorted.to_csv('clean_data/final_combined_df_sorted.csv', index=False)
# final_combined_df_sorted

In [11]:
### Save the merged files


 ### Cleaning the category column 

###  Cleaning the category column 
#### final_combined_df_sorted['Category'].unique() gives a lot of categories, but I weant to clean it so that we only get these: 


#### The desired modified version
      'Posts in Start Here!', 'Posts in From Rural to Urban',
        'Posts in Finding Inspiration', 'Posts in Strategic Studying ',
        'Posts in Wrapping Up', 'Posts in Career Exploration',
        'Posts in Well Being and Self Care',
        'Posts in Post-Secondary & Career Planning',
       'Posts in Confronting Discrimination', 'Posts in Getting Hired',
        'Posts in Dis and Misinformation', 'Posts in Agency in the World',
        'Posts in Paying for school', 'Posts in General Discussion',
        'Posts in "Adulting"'




<!-- 
# ### Original unique values 

# array(['Posts in {"en"=>"Program Instructions"}',
#        'Posts in {:en=>"Post-secondary Considerations - part 1"}',
#        'Posts in {:en=>"Career Considerations - part 1"}',
#        'Posts in {"en"=>"Studying "}',
#        'Posts in {"en"=>"Well-Being and Self Care "}',
#        'Posts in {:en=>"Paying for School - part 1"}',
#        'Posts in {"en"=>"Finding Inspiration"}',
#        'Posts in {"en"=>"Dis- & Misinformation"}',
#        'Posts in {"en"=>"Agency in the world"}',
#        'Posts in {:en=>"Dis/Mis-Information – Ext. Activity: Check Yourself  "}',
#        'Posts in {:en=>"Studying – Ext. Activity: Study Habits"}',
#        'Posts in {"en"=>"Paying for School - part 2"}',
#        'Posts in {:en=>"Well-Being – Ext. Activity: Self Care Plan"}',
#        'Posts in {"en"=>"Post-secondary Considerations - part 2"}',
#        'Posts in {:en=>"Complete Conversation"} Paying for school',
#        'Posts in {"en"=>"From Rural to Urban - part 1"}',
#        'Posts in {"en"=>"From Rural to Urban - part 2"}',
#        'Posts in {"en"=>"Career Considerations - part 2"}',
#        'Posts in {:en=>"Finding Inspiration – Ext Activity: 3 Lines"}',
#        'Posts in {:en=>"Entering the Job Market - part 1"}',
#        'Posts in {:en=>"General Discussion"}',
#        'Posts in {:en=>"Complete Conversation"} Dis-misinformation',
#        'Posts in {:en=>"Well-being – Ext. Activity: Adulting 101"}',
#        'Posts in {:en=>"Wrapping Up"}',
#        'Posts in {:en=>"Studying– Ext. Activity: The Pomodoro Technique"}',
#        'Posts in {:en=>"Complete Conversation"} Well being and self care',
#        'Posts in {"en"=>"Entering the Job Market - part 2"}',
#        'Posts in {:en=>"Ways of Knowing - Part 1"}',
#        'Posts in {"en"=>" Confronting Discrimination"}',
#        'Posts in {:en=>"Take the Survey"}',
#        'Posts in {:en=>"Complete Conversation"} Studying',
#        'Posts in {:en=>"Rural to Urban – Ext. Activity: Keeping Traditions"}',
#        'Posts in {:en=>"Confronting Discrimination – Ext. Activity: Defining Moments"}',
#        'Posts in {:en=>"Complete Conversation"} Rural to urban',
#        'Posts in {:en=>"Entering the Job Market - Ext. Activity: Practice Interview"}',
#        'Posts in {:en=>"Paying for School – Ext. Activity: Make a Budget"}',
#        'Posts in {:en=>"Confronting Discrimination – Ext. Activity: Response Strategies"}',
#        'Posts in {:en=>"Career Considerations – Ext. Activity: Research the Good Life"}',
#        'Posts in {"en"=>"Ways of Knowing - Part 2"}',
#        'Posts in {:en=>"Complete Conversation"} Agency in the World',
#        'Posts in {:en=>"Entering the Job Market - Ext. Activity: Make a Resumé "}',
#        'Posts in {:en=>"Complete Conversation"} Getting Hired',
#        'Posts in {:en=>"Rural to Urban – Ext. Activity: Easing Traditions"}',
#        'Posts in {:en=>"Paying for School – Ext. Activity: Scholarship Applications"}',
#        'Posts in {:en=>"Agency in the World – Ext. Activity: Learning from Others"}',
#        'Posts in {:en=>"Complete Conversation"} Finding Inspiration',
#        'Posts in {:en=>"Ways of Knowing – Ext. Activity: Learning From Stories"}',
#        'Posts in Ways of Knowing',
#        'Posts in {:en=>"Post-secondary Considerations – Ext. Activity: Life Outside of Academics"}',
#        'Posts in {:en=>"Complete Conversation"} Career Considerations',
#        'Posts in Start Here!', 'Posts in From Rural to Urban',
#        'Posts in Finding Inspiration', 'Posts in Strategic Studying ',
#        'Posts in Wrapping Up', 'Posts in Career Exploration',
#        'Posts in Well Being and Self Care',
#        'Posts in Post-Secondary & Career Planning',
#        'Posts in Confronting Discrimination', 'Posts in Getting Hired',
#        'Posts in Dis and Misinformation', 'Posts in Agency in the World',
#        'Posts in Paying for school', 'Posts in General Discussion',
#        'Posts in "Adulting"'], dtype=object) -->

In [12]:
# Mapping dictionary
mapping = {
    'Posts in {"en"=>"Program Instructions"}': 'Posts in Start Here!',
    'Posts in {:en=>"Post-secondary Considerations - part 1"}': 'Posts in Post-Secondary & Career Planning',
    'Posts in {:en=>"Career Considerations - part 1"}': 'Posts in Career Exploration',
    'Posts in {"en"=>"Studying "}': 'Posts in Strategic Studying',
    'Posts in {"en"=>"Well-Being and Self Care "}': 'Posts in Well Being and Self Care',
    'Posts in {:en=>"Paying for School - part 1"}': 'Posts in Paying for school',
    'Posts in {"en"=>"Finding Inspiration"}': 'Posts in Finding Inspiration',
    'Posts in {"en"=>"Dis- & Misinformation"}': 'Posts in Dis and Misinformation',
    'Posts in {"en"=>"Agency in the world"}': 'Posts in Agency in the World',
    'Posts in {:en=>"Dis/Mis-Information – Ext. Activity: Check Yourself  "}': 'Posts in Dis and Misinformation',
    'Posts in {:en=>"Studying – Ext. Activity: Study Habits"}': 'Posts in Strategic Studying',
    'Posts in {"en"=>"Paying for School - part 2"}': 'Posts in Paying for school',
    'Posts in {:en=>"Well-Being – Ext. Activity: Self Care Plan"}': 'Posts in Well Being and Self Care',
    'Posts in {"en"=>"Post-secondary Considerations - part 2"}': 'Posts in Post-Secondary & Career Planning',
    'Posts in {:en=>"Complete Conversation"} Paying for school': 'Posts in Paying for school',
    'Posts in {"en"=>"From Rural to Urban - part 1"}': 'Posts in From Rural to Urban',
    'Posts in {"en"=>"From Rural to Urban - part 2"}': 'Posts in From Rural to Urban',
    'Posts in {"en"=>"Career Considerations - part 2"}': 'Posts in Career Exploration',
    'Posts in {:en=>"Finding Inspiration – Ext Activity: 3 Lines"}': 'Posts in Finding Inspiration',
    'Posts in {:en=>"Entering the Job Market - part 1"}': 'Posts in Getting Hired',
    'Posts in {:en=>"General Discussion"}': 'Posts in General Discussion',
    'Posts in {:en=>"Complete Conversation"} Dis-misinformation': 'Posts in Dis and Misinformation',
    'Posts in {:en=>"Well-being – Ext. Activity: Adulting 101"}': 'Posts in Well Being and Self Care',
    'Posts in {:en=>"Wrapping Up"}': 'Posts in Wrapping Up',
    'Posts in {:en=>"Studying– Ext. Activity: The Pomodoro Technique"}': 'Posts in Strategic Studying',
    'Posts in {:en=>"Complete Conversation"} Well being and self care': 'Posts in Well Being and Self Care',
    'Posts in {"en"=>"Entering the Job Market - part 2"}': 'Posts in Getting Hired',
    'Posts in {:en=>"Ways of Knowing - Part 1"}': 'Posts in Ways of Knowing',
    'Posts in {"en"=>" Confronting Discrimination"}': 'Posts in Confronting Discrimination',
    'Posts in {:en=>"Take the Survey"}': 'Posts in General Discussion',
    'Posts in {:en=>"Complete Conversation"} Studying': 'Posts in Strategic Studying',
    'Posts in {:en=>"Rural to Urban – Ext. Activity: Keeping Traditions"}': 'Posts in From Rural to Urban',
    'Posts in {:en=>"Confronting Discrimination – Ext. Activity: Defining Moments"}': 'Posts in Confronting Discrimination',
    'Posts in {:en=>"Complete Conversation"} Rural to urban': 'Posts in From Rural to Urban',
    'Posts in {:en=>"Entering the Job Market - Ext. Activity: Practice Interview"}': 'Posts in Getting Hired',
    'Posts in {:en=>"Paying for School – Ext. Activity: Make a Budget"}': 'Posts in Paying for school',
    'Posts in {:en=>"Confronting Discrimination – Ext. Activity: Response Strategies"}': 'Posts in Confronting Discrimination',
    'Posts in {:en=>"Career Considerations – Ext. Activity: Research the Good Life"}': 'Posts in Career Exploration',
    'Posts in {"en"=>"Ways of Knowing - Part 2"}': 'Posts in Ways of Knowing',
    'Posts in {:en=>"Complete Conversation"} Agency in the World': 'Posts in Agency in the World',
    'Posts in {:en=>"Entering the Job Market - Ext. Activity: Make a Resumé "}': 'Posts in Getting Hired',
    'Posts in {:en=>"Complete Conversation"} Getting Hired': 'Posts in Getting Hired',
    'Posts in {:en=>"Rural to Urban – Ext. Activity: Easing Traditions"}': 'Posts in From Rural to Urban',
    'Posts in {:en=>"Paying for School – Ext. Activity: Scholarship Applications"}': 'Posts in Paying for school',
    'Posts in {:en=>"Agency in the World – Ext. Activity: Learning from Others"}': 'Posts in Agency in the World',
    'Posts in {:en=>"Complete Conversation"} Finding Inspiration': 'Posts in Finding Inspiration',
    'Posts in {:en=>"Ways of Knowing – Ext. Activity: Learning From Stories"}': 'Posts in Ways of Knowing',
    'Posts in Ways of Knowing': 'Posts in Ways of Knowing',
    'Posts in {:en=>"Post-secondary Considerations – Ext. Activity: Life Outside of Academics"}': 'Posts in Post-Secondary & Career Planning',
    'Posts in {:en=>"Complete Conversation"} Career Considerations': 'Posts in Career Exploration',
    'Posts in "Adulting"': 'Posts in "Adulting"'
}
Master_all_cleaned = pd.read_csv('clean_data/Master_all_Cleaned.csv')

# Replace values using the mapping dictionary
Master_all_cleaned['Category'] = Master_all_cleaned['Category'].map(mapping)

# Get unique values
# Master_all_cleaned['Category'].unique()


In [13]:
# Calculate the number of messages in total
num_messages = len(Master_all_cleaned)
num_messages

11712

In [14]:
# iter = {1,2,3, 4, 5, 6, 7, 9, 34,45}

# for i in iter:
#     print(Master_all_cleaned['Response'][i])
#     print(" ")
#     print(" ")